In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Randomized Algorithm - Selection & Quicksort


### Randomized Algorithms

When we don't know exactly how to proceed in solving a problem, we can make a random choice and hope we made the right one. Alternately, we can view randomization as helping us avoid always making the worst/wrong choice. 


Given an unsorted list $a$ and an integer $k$ ($0\leq k< |a|$), the **order statistics** (or **selection**) problem asks us to return the $k$-th smallest element from $a$.

<p>\begin{array}{ll}  
\mathit{select}~a~k =   
\\  
\texttt{let}  
\\  
~~~~p = \mbox{pick a uniformly random element from}~a
\\  
~~~~\ell = \left\langle\, x \in a \;|\; x < p \,\right\rangle  
\\  
~~~~r = \left\langle\, x \in a \;|\; x > p \,\right\rangle  
\\  
\texttt{in}  
\\  
~~~~\texttt{if}~(k < |\ell|)~\texttt{then}~\mathit{select}~\ell~k  
\\  
~~~~\texttt{else if}~(k < |a| - |r|)~\texttt{then}~p  
\\  
~~~~\texttt{else}~\mathit{select}~r~(k - (|a| - |r|))  
\\  
\texttt{end}  
\end{array}</p>

The recurrence of work is:
$$W(n) = \max\{W(\mid l\mid), W(\mid r\mid)\} + O(n)$$


### Expected Work and Span

**Expectation** of a random variable $X$ defined for a probability space $(\Omega, \mathbf{P})$ is denoted ($\Omega$ is the sample space)

$$ \mathbf{E}\left[{X}\right] = \sum_{x} x \cdot \mathbf{P}\left[X = x\right]. $$

Intuitively, the expectation is really just a weighted average of the values of the random variable.


``Example 1``: Let $X$ be the sum value of a pair of dice. Each trial is independent. $X$ can take on values between 2 and 12, where each has a certain probability based on the possible ways the dice can sum to that value. What is the expected value of the value of a pair of unbiased dice? 


$$\begin{eqnarray*}
\mathbf{E}\left[{X}\right] &=& \sum_{x=2}^{12} x \cdot \mathbf{P}\left[X = x\right] \\
&=& 2\cdot\frac{1}{36} + 3\cdot\frac{2}{36} + 4\cdot\frac{3}{36} + \cdots + 11\cdot\frac{2}{36} + 12\cdot\frac{1}{36}\\
& = & 7
\end{eqnarray*}$$

``Example 2``: Let's look at coin flips where we have equal probability of heads/tails, and define $X$ as the number of flips until we get heads. What is $E[X]$? 

Observe that:

$\begin{eqnarray*}
E[X] &=& \frac{1}{2}1 + \frac{1}{2}(1+E[X]) \\
\end{eqnarray*}$

Solving for $E[X]$, we get that $E[X] = 2$. 

More generally, the expected number of trials to get an outcome of probability $p$ is $1/p$. (We'll see this later.)


The work/span of our algorithm depends on $\max\{W(\mid l\mid), W(\mid r\mid)\}$ in each recursive call. Let $X(n)$ be the fractional size of the larger side of the split, for an input of size $n$:
$$W(n) = \max\{W(\mid l\mid), W(\mid r\mid)\} + O(n)$$
So,

$$X(n) = \frac{\max{\{|l|, |r|\}}}{n}$$

Then the work and span recurrences are:

$$W(n) = W(X(n) \cdot n) + O(n)$$

$$S(n) = S(X(n) \cdot n) + O(\lg n)$$ 



e.g., $n=6$

|i  | len(l) | len(r)  | X(i) | p(i) |
|---|--------|---------|------|------|
|0  | 0      | 5       | 5/6  |  1/6 |
|1  | 1      | 4       | 4/6  |  1/6 |
|2  | 2      | 3       | 3/6  |  1/6 |
|3  | 3      | 2       | 3/6  |  1/6 |
|4  | 4      | 1       | 4/6  |  1/6 |
|5  | 5      | 0       | 5/6  |  1/6 |



First, we'll bound $\mathbf{E}[X(n)].$ If $|l| = i$, then $|r| = n - i -1$. Using the fact that the probability of the pivot being any particular $i$ is $1/n$, we have:

$$\begin{eqnarray*}
 \mathbf{E}\left[{X(n)}\right] &=&  \sum_{i=0}^{n-1}P[X(i)] \cdot X(i)\\
 & = & \frac{1}{n} \sum_{i=0}^{n-1} \frac{\max\{i, n-i-1\}}{n} \\
 &\leq & \frac{1}{n} \sum_{j=n/2}^{n-1} \frac{2}{n} \cdot j  \\
 & \leq & \frac{2}{n^2}\sum_{j=n/2}^{n-1} j =\frac{2}{n^2}\times\frac{3n^2 - 2n}{8}\\
 &\leq & \frac{3}{4}  
\end{eqnarray*}$$

<br>

It might seem tempting to say that we are done. However, we could get "unlucky" in a series of recursions even though $\mathbf{E}[X(n)]\leq 3/4.$ 


$$W(n) \leq W(\frac{3}{4}n) + O(n) \Rightarrow \mathrm{root~dominated}  \Rightarrow W(n)=\mathcal{O}(n)$$

$$S(n) \leq S(\frac{3}{4}n) + O(\lg n) \Rightarrow \mathrm{balanced} \Rightarrow S(n) = \mathcal{O}(\lg^2n)$$ 

What we want to know is:  **What's the probability that the input to the recursive call has size $\le \frac{3}{4} n$ ?**
<br>

We can examine where $p$ might land in the sorted version of $a$, to understand the probability of a good split.

<img width="60%" src="selection-intuition.jpg"/>

If the sampled pivot lies in the green region, then the size of the array passed to the recursive call is at most $3n/4$.

The probability of sampling a point in the green region is $1/2$.

We can see that $\mathbf{P}[\max\{W(\mid l\mid), W(\mid r\mid)\} \leq W(3n/4)] = 1/2$.



If we think of each choice of pivot as a coin flip ("good" vs. "bad") then the expected number of pivot choices to reduce the input to $3n/4$ is 2. 

In other words, every two recursions yields the desired reduction in list size, and so in expectation we will do linear work.

<br><br>
**What if we're unlucky?**

We could keep sampling pivots outside of the green area. What is the probability if we do so $i$ times in a row?

<br><br>
$\frac{1}{2} * \frac{1}{2} * \frac{1}{2} * ... = \frac{1}{2^i}$

E.g., for $i=10$, probability of getting no good pivots is $\approx 0.1\%$. 

Thus, probability of getting at least one good pivot for 10 splits is $\approx 99.9\%$

## Quick Sort

We saw how the problem of selection could be solved with a randomized algorithm. The key was to choose a random element and then partition the list into two parts. 

What if we recursively sorted these two parts?

Let $a=\langle 2, 5, 4, 1, 3, -1, 99\rangle.$ Suppose we chose 4 as the pivot. Then the two parts of the list are $\ell=\langle 2, 1, 3, -1\rangle$ and $r=\langle 5, 99\rangle$. In sorted order they are $\ell'=\langle -1, 1, 2, 3\rangle$ and $r'=\langle 5, 99\rangle$.

So all we have to do is append $l'$, the pivot and $r'$!

This suggests a divide-and-conquer algorithm, but with similar characteristics as our algorithm for selection. 

 <p>\[\begin{array}{ll}  
\mathit{quicksort}~a =  \\  
~~~~\texttt{if}~|a| = 0~\texttt{then}~a  \\  
~~~~\texttt{else}   \\  
~~~~~~~~\texttt{let}  \\  
~~~~~~~~~~~~p = \texttt{pick a random pivot from}~a  \\  
~~~~~~~~~~~~    a_1 = \left\langle\, x \in a \;|\; x < p \,\right\rangle  \\  
~~~~~~~~~~~~    a_2 = \left\langle\, x \in a \;|\; x = p \,\right\rangle  \\  
~~~~~~~~~~~~    a_3 = \left\langle\, x \in a \;|\; x > p \,\right\rangle  \\  
~~~~~~~~~~~~    (s_1,s_3) = (\mathit{quicksort}~a_1)~\mid\mid{}~(\mathit{quicksort}~a_3)  \\  
~~~~~~~~   \texttt{in}  \\  
~~~~~~~~~~~~    s_1 \texttt{++}{} a_2 \texttt{++}{} s_3  \\  
~~~~~~~~  \texttt{end}  
\end{array}\]</p>


Let $X(n)$ be the fractional size of the larger side of the split, for an input of size $n$. So 

$$X(n) = \frac{\max{\{|l|, |r|\}}}{n}$$



Then the work and span recurrences are:
$$W(n) \leq W(X(n) \cdot n)+ W((1-X(n)) \cdot n)+ O(n)$$

$$S(n) \leq S(X(n) \cdot n) + O(\lg n)$$ 

The expected work and span are:

$$W(n) \leq W(\frac{3}{4}n)+W(\frac{1}{4}n) + O(n) \Rightarrow \mathrm{balanced}  \Rightarrow W(n)=\mathcal{O}(n\lg n)$$

$$S(n) \leq S(\frac{3}{4}n) + O(\lg n) \Rightarrow \mathrm{balanced} \Rightarrow S(n) = \mathcal{O}(\lg^2n)$$ 

However, the worse work and span are

$$W(n) \leq W(n-1)+W(1) + O(n) \Rightarrow \mathrm{balanced}  \Rightarrow W(n)=\mathcal{O}(n^2)$$

$$S(n) \leq S(n-1) + O(\lg n) \Rightarrow \mathrm{balanced} \Rightarrow S(n) = \mathcal{O}(n\lg n)$$ 


### Comparison of Sorting Algorithms

<table border="1">
<tr>
<th></th>
        <th colspan="3">Time</th>
        <th colspan="4"></th>
    </tr>
<tr>
<td>Sort</td>
        <td>Average</td>
        <td>Best</td>
        <td>Worst</td>
        <td>Remarks</td>
    </tr>
<tr>
<td><a href="//www.cprogramming.com/tutorial/computersciencetheory/sorting2.html">Selection
                Sort</a></td>
        <td>O(n^2)</td>
        <td>O(n^2)</td>
        <td>O(n^2)</td>
        <td>Even a perfectly sorted input requires scanning the entire array</td>
    </tr>
<tr>
<td><a href="//www.cprogramming.com/tutorial/computersciencetheory/sorting2.html">Insertion
                Sort</a></td>
        <td>O(n^2)</td>
        <td>O(n)</td>
        <td>O(n^2)</td>
        <td>In the best case (already sorted), every insert requires constant time</td>
    </tr>
<tr>
<td><a href="//www.cprogramming.com/tutorial/computersciencetheory/mergesort.html">Merge
                Sort</a></td>
        <td>O(n*log(n))</td>
        <td>O(n*log(n))</td>
        <td>O(n*log(n))</td>
        <td>On arrays, merge sort requires O(n) space; on linked lists, merge
       sort requires constant space</td>
    </tr>
<tr>
<td><a href="//www.cprogramming.com/tutorial/computersciencetheory/quicksort.html">Quicksort</a></td>
        <td>O(n*log(n))</td>
        <td>O(n*log(n))</td>
        <td>O(n^2)</td>
        <td>Randomly picking a pivot value (or shuffling the array prior to
            sorting) can help avoid worst case scenarios such as a perfectly
            sorted array.</td>
    </tr>
</table>